In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import ElasticNet,LinearRegression,Ridge,SGDRegressor,LogisticRegression
from sklearn.metrics import mean_squared_log_error
from sklearn.model_selection import GridSearchCV,cross_val_score,train_test_split
from sklearn.metrics import make_scorer
from sklearn.preprocessing import StandardScaler

In [118]:
path='/Users/qianj/Documents/Kaggle_Competition/PlayGround/Regression with an Abalone Dataset/'
data=pd.read_csv(path+'train.csv')
test=pd.read_csv(path+'test.csv')
data.head()

,id,Sex,Length,Diameter,Height,Whole weight,Whole weight.1,Whole weight.2,Shell weight,Rings
0,0,F,0.550,0.430,0.150,0.7715,0.3285,0.1465,0.2400,11
1,1,F,0.630,0.490,0.145,1.1300,0.4580,0.2765,0.3200,11
2,2,I,0.160,0.110,0.025,0.0210,0.0055,0.0030,0.0050,6
3,3,M,0.595,0.475,0.150,0.9145,0.3755,0.2055,0.2500,10
4,4,I,0.555,0.425,0.130,0.7820,0.3695,0.1600,0.1975,9


In [119]:
data['Sex'].value_counts()
test['Sex'].value_counts()

Sex
I    22241
M    20783
F    17387
Name: count, dtype: int64

In [120]:
train_sex_I=sum(data['Sex']=='I')/len(data)
train_sex_M=sum(data['Sex']=='M')/len(data)
train_sex_F=sum(data['Sex']=='F')/len(data)
test_sex_I=sum(test['Sex']=='I')/len(test)
test_sex_M=sum(test['Sex']=='M')/len(test)
test_sex_F=sum(test['Sex']=='F')/len(test)

In [121]:
data['Sex']=data['Sex'].replace('I',train_sex_I)
data['Sex']=data['Sex'].replace('F',train_sex_F)
data['Sex']=data['Sex'].replace('M',train_sex_M)
test['Sex']=test['Sex'].replace('I',test_sex_I)
test['Sex']=test['Sex'].replace('F',test_sex_F)
test['Sex']=test['Sex'].replace('M',test_sex_M)

/var/folders/46/mjvmwj6x7n5_cdghvmx10bkm0000gn/T/ipykernel_84628/2556581830.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data['Sex']=data['Sex'].replace('M',train_sex_M)
/var/folders/46/mjvmwj6x7n5_cdghvmx10bkm0000gn/T/ipykernel_84628/2556581830.py:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test['Sex']=test['Sex'].replace('M',test_sex_M)


In [124]:
X=data.drop(['id','Rings'],axis=1)
TEST=test.drop(['id'],axis=1)
y=data['Rings']

In [90]:
def RMSLE(yhat,y):
    n=len(yhat)
    a=np.log(1+yhat)
    b=np.log(1+y)
    return np.sqrt(1/n*np.sum((a-b)**2))
rmsle_score=make_scorer(RMSLE,greater_is_better=False)

In [91]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)

In [132]:
n_inputs=X_train.shape[1]
eta=0.001
n_iterations=5001
m=len(X_train)
#epsilon=0.00001
loss=np.inf

Theta=np.ones(n_inputs)

for iteration in range(n_iterations):
    yhat=X_train.dot(Theta)
    loss=RMSLE(yhat,y_train)
    error=yhat-y_train
    gradient=(1/(m*loss)*X_train/(1+X_train*Theta)).T.dot(error)
    Theta=Theta-eta*gradient
print(Theta)

Sex               3.758024
Length            3.907215
Diameter          3.575127
Height            2.236658
Whole weight      3.650145
Whole weight.1    2.216139
Whole weight.2    1.614431
Shell weight      2.245917
dtype: float64


In [133]:
yhat=X_test.dot(Theta).astype(int)
RMSLE(yhat,y_test)

np.float64(0.2774996669692053)

In [134]:
yhat=TEST.dot(Theta).astype(int)

In [92]:
LR=LogisticRegression()
LR.fit(X_train,y_train)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [94]:
yhat=LR.predict(X_test)
RMSLE(yhat,y_test)

np.float64(0.190980113643684)

In [125]:
yhat=LR.predict(TEST)

In [135]:
def file(yhat,test):
    column=pd.DataFrame({'Rings':yhat})
    test=test.join(column).set_index('id')
    test[['Rings']].to_csv(path+'result.csv')
file(yhat,test)